In [4]:
# Mount gdrive

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [6]:
# Change Directory

In [7]:
%cd "/content/drive/My Drive/UCM/tfm_code/notebooks"

/content/drive/My Drive/UCM/tfm_code/notebooks


In [8]:
# Import Libraries

In [9]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 11.3 MB/s eta 0:00:00


In [10]:
# Downloaded
import optuna
import pandas as pd
import pyarrow
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import  StratifiedKFold
import matplotlib.pyplot as plt

In [11]:
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_param_importances

In [12]:
from warnings import filterwarnings
filterwarnings('ignore')

In [13]:
# Codigo

In [14]:
df = pd.read_parquet(engine="pyarrow", path="/content/drive/My Drive/UCM/tfm_code/data/data_modelo_train_cleaned.parquet")

In [15]:
df = df[df['SEMANA'].isin([17,16,15,14])]
df.drop(columns=['CLIENTE_ID','PRODUCTO_ID','ANIO','SEMANA'], inplace=True)

In [16]:
important_columns = ['FRECUENCIA', 'RECENCY', 'TAMANIO', 'UNIDAD_EMPAQUE', 'MARCA_CIEL',
       'MARCA_COCA-COLA', 'MARCA_DEL VALLE', 'MARCA_MUNDET', 'SABOR_COLA',
       'SABOR_NATURAL', 'EMPAQUE_VIDRIO', 'TIPO_CONSUMO_PERSONALES (SS)',
       'CLUSTER_1', 'ROTATION_MEAN_CF', 'ROTATION_MEAN_DAYS',
       'ROTATION_MEDIAN_DAYS', 'CF_LOG']

In [17]:
X = df.drop(columns=['PURCHASE'], axis=1)
X = X[important_columns]
y = df['PURCHASE']

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, stratify=y,random_state=42)

In [18]:
# del(df)
# del(X)
# del(y)
# del(X_train)
# del(y_train)

In [19]:
np.random.seed(1234)

In [20]:
# Split your data into features (X) and target variable (y)
X_train, X, y_train, y = train_test_split(X, y, test_size=0.14, stratify=y,random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y,random_state=42)

In [21]:
X_train['FRECUENCIA'].count()

211164

In [22]:
def objective(trial):
    params = {
        'n_jobs': -1,
        'max_depth': trial.suggest_int('max_depth', 4, 12),
        'max_features': trial.suggest_uniform('max_features', 0.4, 1.0),
        'min_samples_split': trial.suggest_int('min_samples_split', 10, 30),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 10, 30),
        'bootstrap': trial.suggest_categorical('bootstrap',  [True,False]),
        'n_estimators': trial.suggest_int('n_estimators', 100, 300),
        'random_state': 42
    }
    # Train and evaluate the model using the given hyperparameters
    clf = RandomForestClassifier(**params)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    f1 = f1_score(y_test, y_pred)
    return f1

# Run hyperparameter tuning with Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Print the best hyperparameters and the corresponding ROC AUC score
print('Best F1-Score:', study.best_value)
print('Best hyperparameters:', study.best_params)

[I 2023-03-17 14:03:25,062] A new study created in memory with name: no-name-788711a1-b47d-4109-9813-d72bf5e37006
[I 2023-03-17 14:05:00,297] Trial 0 finished with value: 0.5594371120622045 and parameters: {'max_depth': 7, 'max_features': 0.8215592827207849, 'min_samples_split': 20, 'min_samples_leaf': 24, 'bootstrap': False, 'n_estimators': 198}. Best is trial 0 with value: 0.5594371120622045.
[I 2023-03-17 14:05:38,960] Trial 1 finished with value: 0.5603185967424378 and parameters: {'max_depth': 7, 'max_features': 0.8431204241565017, 'min_samples_split': 28, 'min_samples_leaf': 12, 'bootstrap': True, 'n_estimators': 111}. Best is trial 1 with value: 0.5603185967424378.
[I 2023-03-17 14:06:23,042] Trial 2 finished with value: 0.5598922679833913 and parameters: {'max_depth': 10, 'max_features': 0.5117721920658902, 'min_samples_split': 24, 'min_samples_leaf': 20, 'bootstrap': True, 'n_estimators': 139}. Best is trial 1 with value: 0.5603185967424378.
[I 2023-03-17 14:08:16,905] Trial 3

Best F1-Score: 0.5618178165665945
Best hyperparameters: {'max_depth': 11, 'max_features': 0.8049113435648149, 'min_samples_split': 14, 'min_samples_leaf': 11, 'bootstrap': False, 'n_estimators': 172}


In [23]:
plot_optimization_history(study)

In [24]:
plot_param_importances(study)